In [52]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [53]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv')


In [54]:
# Preprocessing

df = df.fillna(0)
all_train_df, test_df = train_test_split(df, test_size=0.2, random_state=1)
train_df, val_df = train_test_split(all_train_df, test_size=0.25, random_state=1)

y_train = train_df['fuel_efficiency_mpg'].values
y_all_train = all_train_df['fuel_efficiency_mpg'].values
y_val = val_df['fuel_efficiency_mpg'].values
y_test = test_df['fuel_efficiency_mpg'].values
del train_df['fuel_efficiency_mpg']
del all_train_df['fuel_efficiency_mpg']
del val_df['fuel_efficiency_mpg']
del test_df['fuel_efficiency_mpg']

train_dict = train_df.to_dict(orient='records')
all_train_dict = all_train_df.to_dict(orient='records')
val_dict = val_df.to_dict(orient='records')
test_dict = test_df.to_dict(orient='records')

dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dict)
X_val = dv.fit(val_dict)
X_all_train= dv.fit(all_train_dict)
X_test = dv.fit_transform(test_dict)



# Question 1
Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable.

Train a model with `max_depth=1`.
Which feature is used for splitting the data?

- 'vehicle_weight'
- 'model_year'
- 'origin'
- 'fuel_type'

In [55]:
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train,y_train)

print(export_text(dt))

|--- feature_13 <= 3022.11
|   |--- value: [16.88]
|--- feature_13 >  3022.11
|   |--- value: [12.94]

